# TD et TME 8: CREATION DE SCHEMAS-CONTRAINTES D'INTEGRITE


## Installation H2 

Le système utilisé pendant les TME est H2.

In [1]:
pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Télécharger le pilote de H2

In [2]:
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/postgresql-42.6.0.jar -O postgresql-42.6.0.jar
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/h2-2.1.214.jar -O h2-2.1.214.jar

--2024-04-14 17:46:23--  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/postgresql-42.6.0.jar
Resolving nuage.lip6.fr (nuage.lip6.fr)... 132.227.201.11
Connecting to nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1081604 (1,0M) [application/octet-stream]
Saving to: ‘postgresql-42.6.0.jar’

postgresql-42.6.0.j 100%[===================>]   1,03M  --.-KB/s    in 0,009s  

2024-04-14 17:46:23 (114 MB/s) - ‘postgresql-42.6.0.jar’ saved [1081604/1081604]

--2024-04-14 17:46:23--  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/h2-2.1.214.jar
Resolving nuage.lip6.fr (nuage.lip6.fr)... 132.227.201.11
Connecting to nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2543011 (2,4M) [application/octet-stream]
Saving to: ‘h2-2.1.214.jar’

h2-2.1.214.jar      100%[===================>]   2,42M  --.-KB/s    in 0,02s   

2024-04-14 17:46:2

**Relancez le kernel**: Kernel -> Restart kernel ...

https://www.psycopg.org/docs/
http://localhost:8082

In [3]:
import psycopg2
import os
local_dir = "./data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

[]

### Démarrer le serveur SQL H2

In [6]:
#Remplacez les valeurs des variables suivantes
#NN= numéro de groupe: chaine de caractères (ex: 01, 02,.., 13), remplacer xx 
NN = '06'
#BB=numero binome: chaîne de caractères (assigné par l'enseignant, ex: 01,..., 15), remplacer xx
BB = '13'
port = 5000+int(NN+BB)
print(f'Le numero du port utilisé pour la connexion à la BD est: {port}')

Le numero du port utilisé pour la connexion à la BD est: 5613


In [7]:
%%bash --bg --out output -s "$port"
java -Dh2.bindAddress=127.0.0.1 -cp h2-2.1.214.jar:postgresql-42.6.0.jar org.h2.tools.Server -pg -pgPort $1 -baseDir ./data -ifNotExists

## Fonctions utiles

### connect_H2

In [8]:
def connect_H2(db,user,password):
    global connection
    try:
        connection
    except:
        connection = None
    if connection != None:
        try:
            connection.close()
            print("Connection closed")
        except  Error as e:
            print(f"The error '{e}' occurred")
    try:
        # connection = sqlite3.connect(path,isolation_level='DEFERRED')
        connection = psycopg2.connect(f"dbname={db} user={user} password={password} host=localhost port={port}")
        print("Connection to H2 DB successful")
    except Exception as e:
        print(f"The error '{e}' occurred")
    return connection

### execute

In [9]:
def execute(connection, query, show=True,close=True):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        print(cursor.rowcount,"rows")
        if show and cursor.rowcount and cursor.description is not None:
            names = [desc[0] for desc in cursor.description]
            lengths={}
            for attr in names:
                lengths[attr]=len(attr)
            for ligne in cursor:
                i=0
                for attr in ligne:
                    lengths[names[i]]=max(lengths[names[i]],len(str(attr).replace('\n',' ')))
                    i=i+1
            print('|',end='')
            for attr in names:
                print(str(attr).ljust(lengths[attr]),end='|')
            print()
            print('|',end='')
            for attr in names:
                print(''.ljust(lengths[attr]+1,'-'),end='')
            print()
            cursor.execute(query)
            for ligne in cursor:
                i=0
                print('|',end='')
                for attr in ligne:
                    print(str(attr).replace('\n',' ')[:lengths[names[i]]].ljust(lengths[names[i]]),end='|')
                    i=i+1
                print()
        if close:
            cursor.close()
    except Exception as e:
        print(f"The error '{e}' occurred")
        cursor=None
    return cursor

### show_table

In [10]:
def show_table(connection,table_name):

    query=f"""
SELECT table_name,
       column_name,
       data_type,
       column_default,
       is_nullable,
       character_maximum_length,
       numeric_precision
FROM information_schema.columns 
where lower(table_name)  = '{table_name}'  
"""
    execute(connection,query)
    

### show_schema

In [11]:
def show_schema(connection):

    print('*********** Tables ************')
    query="""
    select TABLE_NAME
    from INFORMATION_SCHEMA.TABLES 
    where TABLE_SCHEMA = 'public'
    """
    execute(connection,query,show=True)

    print('\n\n*********** Domaines ************')
    query="""
    SELECT domain_name,check_clause 
    FROM information_schema.domain_constraints a, information_schema.check_constraints b
    where a.constraint_name=b.constraint_name
    """
    execute(connection,query)

    print('\n\n*********** Attributs ************')
    query=f"""
    SELECT c.table_name,
           c.column_name,
           c.data_type,
           c.column_default,
           c.is_nullable,
           c.character_maximum_length,
           c.numeric_precision
    FROM INFORMATION_SCHEMA.TABLES t, information_schema.columns c
    where t.table_name=c.table_name
    and t.TABLE_SCHEMA = 'public'
    """
    execute(connection,query)
    

## Connexion à H2

In [12]:
connection = connect_H2("jo"+f'{port}',"ba","ba")

Connection to H2 DB successful


## Exercice 1 : Création de schéma 
Créer le schéma préparé en TD

On considère le schéma Entreprise décrit ci-dessous.

- EMPLOYE (<u>NumSS</u>, NomE, PrenomE, NumChef*, VilleE, DateNaiss) 
- PROJET(<u>NumProj</u>, NomProj, RespProj*, VilleP, Budget) 
- EMBAUCHE (<u>NumSS*, NumProj*</u>, DateEmb, Profil*) 
- GRILLE_SAL (<u>Profil</u>, salaire)

La clé primaire de chaque relation est soulignée et les attributs des clés étrangères sont suivis d’un astérisque. Cette base contient des informations sur des employés et sur les projets dans lesquels ils sont impliqués. Ces employés sont embauchés dans un projet sur un profil donné et perçoivent un salaire en fonction de ce profil. Le chef d'un employé dans la table Employé et le chef d'un projet dans la table Projet sont des employés. En plus des contraintes de clé et des contraintes référentielles indiquées dans le schéma, on voudrait intégrer les contraintes suivantes :

Domaines
- Le numéro de sécurité sociale possède exactement 5 chiffres.
- Les attributs textuels (NomE, PrenomE, NomProj, Profil) ne dépassent pas 20 caractères (ils peuvent en avoir moins).
- La ville d’un employé (VillE) ou d’un projet (VilleP) se limite à 'Paris', 'Lyon' et 'Marseille' et sa longueur ne dépasse pas 9 caractères.
- Le numéro d’un projet varie entre 5 et 7 chiffres.
- Le salaire peut avoir deux chiffres après la virgule et ne dépasse pas 90 000.
- Le budget est un entier sur 6 chiffres (sans virgule)
- Aucun employé ne peut avoir plus de 70 ans au moment où il est enregistré dans la table Employé.

Clés
- Il n'y a pas deux employés avec le même nom et le même prénom.

Not-null
- Chaque projet doit avoir un responsable.
- Dans la table Embauche, NumSS et NumProj représentent un employé et un projet existants. De même, profil est un profil existant.



Donner en SQL les instructions de création des contraintes de domaines indiquées dans l'énoncé.

#### Supprimer tout le schéma de la base courante


In [25]:
query="""
DROP ALL OBJECTS
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7fcae0cbe400; closed: -1>

#### Définition des domaines

Le numéro de sécurité sociale possède exactement 5 chiffres

In [26]:
query="""
create domain dnumss as numeric(5) check (length(value)=5)
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7fcae0135d60; closed: -1>

Les attributs textuels (NomE, PrenomE, NomProj, Profil) ne dépassent pas 20 caractères (ils peuvent en avoir moins)

In [27]:
query="""
create domain dchaines as varchar(20)
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7fcae0135e50; closed: -1>

La ville d’un employé (VilleE) ou d’un projet (VilleP) se limite à 'Paris', 'Lyon' et 'Marseille' et sa longueur ne dépasse pas 9 caractères

In [28]:
query="""
create domain dvilles as varchar(9) check(lower(value) IN ('paris', 'lyon', 'marseille'))
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7fcae0cbe310; closed: -1>

Le numéro d’un projet varie entre 5 et 7 chiffres.

In [29]:
query="""
create domain dnumproj as numeric(7) check(length(value)>4)
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7fcae0cbe7c0; closed: -1>

Le salaire peut avoir deux chiffres après la virgule et ne dépasse pas 90 000.

In [32]:
query="""
create domain dsal as numeric(7,2) check(value<=90000)
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7fcae0cbe6d0; closed: -1>

Budget sur 6 chiffres

In [33]:
query="""
create domain dbudget as numeric(6) check(length(value) = 6)
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7fcae0cbe9a0; closed: -1>

Aucun employé ne peut avoir plus de 70 ans au moment où il est enregistré dans la table Employé.

In [34]:
query="""
create domain ddatenaiss as date check(datediff(year,value,current_date)<=70)
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7fcae0cbea90; closed: -1>

#### Interroger le catalogue pour vérifier les domaines existants

In [35]:
query="""
select domain_name 
from INFORMATION_SCHEMA.DOMAINS 
"""
execute(connection,query,show=True)

7 rows
|domain_name|
|------------
|dvilles    |
|dbudget    |
|dnumproj   |
|ddatenaiss |
|dchaines   |
|dsal       |
|dnumss     |


<cursor object at 0x7fcae0cbeb80; closed: -1>

In [36]:
query="""
SELECT domain_name,check_clause 
FROM information_schema.domain_constraints a, information_schema.check_constraints b
where a.constraint_name=b.constraint_name
"""
execute(connection,query)

6 rows
|domain_name|check_clause                                 |
|----------------------------------------------------------
|dsal       |VALUE <= 90000                               |
|dbudget    |CHAR_LENGTH(VALUE) = 6                       |
|dvilles    |LOWER(VALUE) IN('paris', 'lyon', 'marseille')|
|dnumproj   |CHAR_LENGTH(VALUE) > 4                       |
|dnumss     |CHAR_LENGTH(VALUE) = 5                       |
|ddatenaiss |DATEDIFF(YEAR, VALUE, CURRENT_DATE) <= 70    |


<cursor object at 0x7fcae0cbec70; closed: -1>

Donner en SQL les instructions de création du schéma de la base en leur associant les contraintes de clés, référentielles et de domaines indiquées dans l'énoncé.

#### Définition des tables

Table Employe

In [40]:
query="""
create table Employe(
    numss dnumss,
    nomE dchaines,
    prenomE dchaines,
    numChef dnumSS,
    villeE dvilles,
    dateNaiss ddatenaiss,
    constraint pk_emp primary key(numss),
    constraint fk_chef foreign key (numChef) references Employe(numss),
    constraint unique_nom_prenom unique(nomE, prenomE) 
)
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7fcae0cbe8b0; closed: -1>

In [42]:
show_table(connection,'Employe')

0 rows


Table Projet

In [50]:
query="""
create table Projet(
    numProj dnumproj,
    nomProj dchaines,
    respProj dnumss not null,
    villeP dvilles,
    budget dbudget,
    constraint pk_proj primary key(numproj),
    constraint fk_respProj foreign key(respProj) references Employe
)
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7fcaf030f5e0; closed: -1>

In [51]:
show_table(connection,'Projet')

0 rows


Table Grille_sal

In [52]:
query="""
create table Grille_sal(
    profile dchaines,
    salaire dsal,
    constraint pk_gril primary key (profile)
)
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7fcaf030f220; closed: -1>

Table Embauche

In [76]:
query="""
create table Embauche(
    numss dnumss,
    numProj dnumproj,
    dateEmb date,
    profil dchaines not null,
    constraint pk_emb primary key(numSS, numProj),
    constraint fk_numss foreign key(numss) references Employe,
    constraint fk_numproj foreign key(numProj) references Projet,
    constraint fk_prof foreign key(profil) references Grille_sal
)
"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7fcae01354f0; closed: -1>

#### Alter table (ajouter des contraintes) 

En considérant que la table projet a été créée, dans ce cas utiliser la commande 

    alter table <table> add constraint <nom><expression>

pour associer la contrainte à Projet

Un responsable de projet doit habiter la ville du projet dont il est responsable. Donner en SQL l'instruction permettant d’exprimer cette contrainte de deux manière différentes

In [77]:
query="""
alter table PROJET add constraint ck_respProjet check(respProj in (select numss from employe where villeE = villep))
"""
execute(connection,query,show=True)

The error 'La contrainte "ck_respprojet" existe déjà
Constraint "ck_respprojet" already exists; SQL statement:

alter table PROJET add constraint ck_respProjet check(respProj in (select numss from employe where villeE = villep) [90045-214]
DETAIL:  org.h2.jdbc.JdbcSQLSyntaxErrorException: La contrainte "ck_respprojet" existe déjà
Constraint "ck_respprojet" already exists; SQL statement:

alter table PROJET add constraint ck_respProjet check(respProj in (select numss from employe where villeE = villep) [90045-214]
' occurred


#### Interroger le catalogue de données

In [78]:
query="""
select TABLE_NAME
from INFORMATION_SCHEMA.TABLES 
where TABLE_SCHEMA = 'public'
"""
execute(connection,query,show=True)

4 rows
|table_name|
|-----------
|embauche  |
|employe   |
|projet    |
|grille_sal|


<cursor object at 0x7fcaf030fe50; closed: -1>

In [79]:
query="""
SELECT t.table_name,constraint_name 
FROM information_schema.table_constraints t
"""
execute(connection,query,show=True)

11 rows
|table_name|constraint_name  |
|-----------------------------
|employe   |unique_nom_prenom|
|projet    |fk_respproj      |
|grille_sal|pk_gril          |
|employe   |pk_emp           |
|projet    |ck_respprojet    |
|embauche  |fk_prof          |
|embauche  |pk_emb           |
|employe   |fk_chef          |
|projet    |pk_proj          |
|embauche  |fk_numss         |
|embauche  |fk_numproj       |


<cursor object at 0x7fcae0135310; closed: -1>

In [80]:
query="""
SELECT t.table_name,constraint_name,column_name,data_type 
FROM information_schema.table_constraints t, information_schema.columns c
where t.table_name=c.table_name
"""
execute(connection,query,show=True)

51 rows
|table_name|constraint_name  |column_name|data_type        |
|-----------------------------------------------------------
|employe   |unique_nom_prenom|numss      |numeric          |
|employe   |unique_nom_prenom|nome       |character varying|
|employe   |unique_nom_prenom|prenome    |character varying|
|employe   |unique_nom_prenom|numchef    |numeric          |
|employe   |unique_nom_prenom|villee     |character varying|
|employe   |unique_nom_prenom|datenaiss  |date             |
|projet    |fk_respproj      |numproj    |numeric          |
|projet    |fk_respproj      |nomproj    |character varying|
|projet    |fk_respproj      |respproj   |numeric          |
|projet    |fk_respproj      |villep     |character varying|
|projet    |fk_respproj      |budget     |numeric          |
|grille_sal|pk_gril          |profile    |character varying|
|grille_sal|pk_gril          |salaire    |numeric          |
|employe   |pk_emp           |numss      |numeric          |
|employe   |pk_e

<cursor object at 0x7fcae01356d0; closed: -1>

In [81]:
show_schema(connection)

*********** Tables ************
4 rows
|table_name|
|-----------
|embauche  |
|employe   |
|projet    |
|grille_sal|


*********** Domaines ************
6 rows
|domain_name|check_clause                                 |
|----------------------------------------------------------
|dsal       |VALUE <= 90000                               |
|dbudget    |CHAR_LENGTH(VALUE) = 6                       |
|dvilles    |LOWER(VALUE) IN('paris', 'lyon', 'marseille')|
|dnumproj   |CHAR_LENGTH(VALUE) > 4                       |
|dnumss     |CHAR_LENGTH(VALUE) = 5                       |
|ddatenaiss |DATEDIFF(YEAR, VALUE, CURRENT_DATE) <= 70    |


*********** Attributs ************
17 rows
|table_name|column_name|data_type        |column_default|is_nullable|character_maximum_length|numeric_precision|
|---------------------------------------------------------------------------------------------------------------
|embauche  |numss      |numeric          |None          |NO         |None                

## Exercice 2 : Insertion de données

Dans ce qui suit, il vous est demandé d'effectuer des insertions de n-uplets dans des tables.

Syntaxe des insertions :

    insert into Table values ('val1', 'val2', ….)

par exemple, pour insérer une employée 'LARS Anna', qui habite 'Paris' et qui est née le 25-08-1975, il suffit d'exécuter l'instruction ci-dessous

    insert into employe values('21456','LARS', 'Anna',null, 'paris', parsedatetime('25-08-1975', 'dd-MM-yyyy' ))
    
et de constater que la système retourne bien le message

    1 rows


Syntaxe des suppressions :

    delete from Table 

supprime tous les nuplets de la table (la table existera toujours mais sera vide)


### Insérez dans chaque table au moins un n-uplet qui vérifie les contraintes d'intégrité. Vous avez la liberté de choisir les valeurs que vous voulez. Par exemple, l'instruction suivante insère un employé qui vérifie les contraintes

In [82]:
query="""
insert into employe values ('21456','LARS', 'Anna',null, 'paris', parsedatetime('25-08-1975', 'dd-MM-yyyy' ))
insert into Employe(numSS, nomE, prenomE, villeE, dateNaiss) values(12345, 'Smith', 'John', 'Paris', parsedatetime('25-08-1975', 'dd-MM-yyyy'))

insert into Projet values(54321, 'Projet-v0', 12345, 'Paris', 100000)

insert into Grille_Sal values('manager', 4000)

insert into Embauche values(12345, 54321, '2023-03-03', 'manager')
"""
execute(connection,query,show=True)

1 rows


<cursor object at 0x7fcaf030fd60; closed: -1>

In [83]:
query="""
select * from employe
"""
execute(connection,query,show=True)

2 rows
|numss|nome |prenome|numchef|villee|datenaiss |
|----------------------------------------------
|21456|LARS |Anna   |None   |paris |1975-08-25|
|12345|Smith|John   |None   |Paris |1975-08-25|


<cursor object at 0x7fcae0cbed60; closed: -1>

In [87]:
query="""
select * from projet
"""
execute(connection,query,show=True)

1 rows
|numproj|nomproj  |respproj|villep|budget|
|-----------------------------------------
|54321  |Projet-v0|12345   |Paris |100000|


<cursor object at 0x7fcacbbe2040; closed: -1>

In [88]:
query="""
select * from grille_sal
"""
execute(connection,query,show=True)

1 rows
|profile|salaire|
|----------------
|manager|4000.00|


<cursor object at 0x7fcacbbe2130; closed: -1>

In [89]:
query="""
select * from embauche
"""
execute(connection,query,show=True)

1 rows
|numss|numproj|dateemb   |profil |
|---------------------------------
|12345|54321  |2023-03-03|manager|


<cursor object at 0x7fcacbbe2220; closed: -1>

## Exercice 3 : Violation des contraintes

### Proposez des insertions qui violent les contraintes d'intégrité définies pour chaque table. 

In [90]:
query="""
select * from employe
"""
execute(connection,query,show=True)

2 rows
|numss|nome |prenome|numchef|villee|datenaiss |
|----------------------------------------------
|21456|LARS |Anna   |None   |paris |1975-08-25|
|12345|Smith|John   |None   |Paris |1975-08-25|


<cursor object at 0x7fcacbbe2310; closed: -1>

Proposer une insertion dans la table Employé qui ne respecte pas la contrainte de clé primaire.

In [91]:
query="""
insert into Employe values(12345, 'Dupont', 'Alice', null, 'Marseille', '2003-10-23')
"""
execute(connection,query,show=True)

The error 'Violation d'index unique ou clé primaire: "public.PRIMARY_KEY_9 ON public.employe(numss) VALUES ( /* 3 */ CAST(12345 AS NUMERIC(5)) )"
Unique index or primary key violation: "public.PRIMARY_KEY_9 ON public.employe(numss) VALUES ( /* 3 */ CAST(12345 AS NUMERIC(5)) )"; SQL statement:

insert into Employe values(12345, 'Dupont', 'Alice', null, 'Marseille', '2003-10-23' [23505-214]
DETAIL:  org.h2.jdbc.JdbcSQLIntegrityConstraintViolationException: Violation d'index unique ou clé primaire: "public.PRIMARY_KEY_9 ON public.employe(numss) VALUES ( /* 3 */ CAST(12345 AS NUMERIC(5)) )"
Unique index or primary key violation: "public.PRIMARY_KEY_9 ON public.employe(numss) VALUES ( /* 3 */ CAST(12345 AS NUMERIC(5)) )"; SQL statement:

insert into Employe values(12345, 'Dupont', 'Alice', null, 'Marseille', '2003-10-23' [23505-214]
' occurred


Proposer une insertion dans la table Employe qui ne respecte pas la contrainte de limite d'âge.

In [92]:
query="""
insert into Employe values(67890, 'Poe', 'John', null, 'Marseille', '1952-10-23')
"""
execute(connection,query,show=True)

The error 'Vérifiez la violation de contrainte: "(DATEDIFF(YEAR, VALUE, CURRENT_DATE) <= 70)"
Check constraint violation: "(DATEDIFF(YEAR, VALUE, CURRENT_DATE) <= 70)"; SQL statement:

insert into Employe values(67890, 'Poe', 'John', null, 'Marseille', '1952-10-23' [23513-214]
DETAIL:  org.h2.jdbc.JdbcSQLIntegrityConstraintViolationException: Vérifiez la violation de contrainte: "(DATEDIFF(YEAR, VALUE, CURRENT_DATE) <= 70)"
Check constraint violation: "(DATEDIFF(YEAR, VALUE, CURRENT_DATE) <= 70)"; SQL statement:

insert into Employe values(67890, 'Poe', 'John', null, 'Marseille', '1952-10-23' [23513-214]
' occurred


Proposer une insertion dans la table Employe qui ne respecte pas la contrainte de longueur de l'attribut NumSS.

In [93]:
query="""
insert into Employe values(123, 'Dupont', 'Alice', null, 'Marseille', '2003-10-23')
"""
execute(connection,query,show=True)

The error 'Vérifiez la violation de contrainte: "(CHAR_LENGTH(VALUE) = 5)"
Check constraint violation: "(CHAR_LENGTH(VALUE) = 5)"; SQL statement:

insert into Employe values(123, 'Dupont', 'Alice', null, 'Marseille', '2003-10-23' [23513-214]
DETAIL:  org.h2.jdbc.JdbcSQLIntegrityConstraintViolationException: Vérifiez la violation de contrainte: "(CHAR_LENGTH(VALUE) = 5)"
Check constraint violation: "(CHAR_LENGTH(VALUE) = 5)"; SQL statement:

insert into Employe values(123, 'Dupont', 'Alice', null, 'Marseille', '2003-10-23' [23513-214]
' occurred


Proposer une insertion dans la table Employe qui ne respecte pas la contrainte sur les villes possibles.

In [94]:
query="""
insert into Employe values(67890, 'Dupont', 'Alice', null, 'Caen', '2003-10-23')
"""
execute(connection,query,show=True)

The error 'Vérifiez la violation de contrainte: "(LOWER(VALUE) IN('paris', 'lyon', 'marseille'))"
Check constraint violation: "(LOWER(VALUE) IN('paris', 'lyon', 'marseille'))"; SQL statement:

insert into Employe values(67890, 'Dupont', 'Alice', null, 'Caen', '2003-10-23' [23513-214]
DETAIL:  org.h2.jdbc.JdbcSQLIntegrityConstraintViolationException: Vérifiez la violation de contrainte: "(LOWER(VALUE) IN('paris', 'lyon', 'marseille'))"
Check constraint violation: "(LOWER(VALUE) IN('paris', 'lyon', 'marseille'))"; SQL statement:

insert into Employe values(67890, 'Dupont', 'Alice', null, 'Caen', '2003-10-23' [23513-214]
' occurred


Insérer dans la table Employé deux employés avec le même nom et le même prénom.

In [95]:
query="""
insert into Employe values(67890, 'Smith', 'John', null, 'Marseille', '2003-10-23')
"""
execute(connection,query,show=True)

The error 'Violation d'index unique ou clé primaire: "public.unique_nom_prenom_INDEX_9 ON public.employe(nome NULLS LAST, prenome NULLS LAST) VALUES ( /* key:3 */ 'Smith', 'John')"
Unique index or primary key violation: "public.unique_nom_prenom_INDEX_9 ON public.employe(nome NULLS LAST, prenome NULLS LAST) VALUES ( /* key:3 */ 'Smith', 'John')"; SQL statement:

insert into Employe values(67890, 'Smith', 'John', null, 'Marseille', '2003-10-23' [23505-214]
DETAIL:  org.h2.jdbc.JdbcSQLIntegrityConstraintViolationException: Violation d'index unique ou clé primaire: "public.unique_nom_prenom_INDEX_9 ON public.employe(nome NULLS LAST, prenome NULLS LAST) VALUES ( /* key:3 */ 'Smith', 'John')"
Unique index or primary key violation: "public.unique_nom_prenom_INDEX_9 ON public.employe(nome NULLS LAST, prenome NULLS LAST) VALUES ( /* key:3 */ 'Smith', 'John')"; SQL statement:

insert into Employe values(67890, 'Smith', 'John', null, 'Marseille', '2003-10-23' [23505-214]
' occurred


Proposer une insertion dans la table Grille_SAL qui ne respecte pas la contrainte sur le salaire qui ne doit pas dépasser 90 000.

In [98]:
query="""
insert into Grille_SAL values('directeur', 100000)
"""
execute(connection,query,show=True)

The error 'Valeur trop longue pour la colonne "salaire NUMERIC(7, 2)": "100000 (32)"
Value too long for column "salaire NUMERIC(7, 2)": "100000 (32)"; SQL statement:

insert into Grille_SAL values('directeur', 100000 [22001-214]
DETAIL:  org.h2.jdbc.JdbcSQLDataException: Valeur trop longue pour la colonne "salaire NUMERIC(7, 2)": "100000 (32)"
Value too long for column "salaire NUMERIC(7, 2)": "100000 (32)"; SQL statement:

insert into Grille_SAL values('directeur', 100000 [22001-214]
' occurred


Proposer une insertion dans la table Projet qui ne respecte pas la contrainte référentielle vers Employe : insérer un responsable de projet qui n'est pas dans la table Employé

In [99]:
query=""" 
insert into Projet values(9876543, 'AlphaGO', 13579, 'Paris', 100000)
"""
execute(connection,query,show=True)

The error 'Vérifiez la violation de contrainte: "ck_respprojet: "
Check constraint violation: "ck_respprojet: "; SQL statement:
 
insert into Projet values(9876543, 'AlphaGO', 13579, 'Paris', 10000 [23513-214]
DETAIL:  org.h2.jdbc.JdbcSQLIntegrityConstraintViolationException: Vérifiez la violation de contrainte: "ck_respprojet: "
Check constraint violation: "ck_respprojet: "; SQL statement:
 
insert into Projet values(9876543, 'AlphaGO', 13579, 'Paris', 10000 [23513-214]
' occurred


Il est possible que la contrainte que le responsable soit dans la même ville est déclenchée. Vous pouvez enlever cette contrainte avec la commande :

    alter table drop constraint "nom de contrainte"
    
Il faudrait la recréer après avec

    alter table add constraint ....

Proposer une insertion dans la table Embauche qui ne respecte pas une des contraintes référentielles : par exemple, associer un employé existant à un projet qui n'existe pas.

In [100]:
query="""
insert into Embauche values values(12345, 9876543, '2023-01-10', 'manager');
"""
execute(connection,query,show=True)

The error 'Le nombre de colonnes ne correspond pas
Column count does not match; SQL statement:
insert into Embauche values values(12345, 9876543, '2023-01-10', 'manager') [21002-214]
DETAIL:  org.h2.jdbc.JdbcSQLSyntaxErrorException: Le nombre de colonnes ne correspond pas
Column count does not match; SQL statement:
insert into Embauche values values(12345, 9876543, '2023-01-10', 'manager') [21002-214]
' occurred


# Fermer la connexion

In [101]:
connection.commit() # implicit avec close
connection.close()